# Neuronpedia_followup

Here are two possibly-weird things I noticed when playing with the Bloom SAEs on Neuronpedia:

1. There are 'starts with letter k' and 'starts with letter z' features that propagate down through the layers, starting with layer 1. (0-indexed). Possibly this has an obvious root cause, but I don't currently understand why this would be. Can we automatically identify these features and group them? Possibly this makes the space of features a human must crawl smaller.
2. Lots of the layer-0 features have weird top postive logits: GoldMagikarp (a known tokenizer thing), but also Lumpur, CLASSIFIED, NetMessage. What's going on? 

### Preamble

In [3]:
import numpy as np
import torch

from huggingface_hub import hf_hub_download
from sae_lens.training.session_loader import LMSparseAutoencoderSessionloader

### Attempting to group and track common features across layers 

In [4]:
pass

### Looking at why these strange tokens appear so frequently

Most notably, CLASSIFIED, NetMessage, largeDownload, GoldMagikarp, Lumpur.

Googling reveals known weirdness with some of these: https://www.lesswrong.com/posts/aPeJE8bSo6rAFoLqg/solidgoldmagikarp-plus-prompt-generation. Notably with longer versions of these words.

Update - lots of these are indeed 'weird tokens' - natureconservancy, NetMessage, Streamer, GoldMagikarp, largeDownload, NetMessage. But regardless, learning why it's always them coming up will be useful, and they should probably be excluded, for no other reason than because they destroy the autointerps

In [6]:
# lifting from https://github.com/jbloomAus/SAELens/blob/main/tutorials/evaluating_your_sae.ipynb

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

torch.set_grad_enabled(False)


In [7]:
# Download the SAE for layer 0 (the one with the Magikarps)

REPO_ID = "jbloom/GPT2-Small-SAEs"
layer = 0  # any layer from 0 - 11 works here
FILENAME = f"final_sparse_autoencoder_gpt2-small_blocks.{layer}.hook_resid_pre_24576.pt"

path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)

In [8]:
# We can then load the SAE, dataset and model using the session loader
model, sparse_autoencoders, activation_store = (
    LMSparseAutoencoderSessionloader.load_session_from_pretrained(path=path)
)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cuda


/home/will/miniconda3/envs/mechinterp/lib/python3.11/site-packages/datasets/load.py:1461: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Run name: 24576-L1-8e-05-LR-0.0004-Tokens-3.000e+08
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.004096
Total training steps: 73242
Total wandb updates: 732
n_tokens_per_feature_sampling_window (millions): 524.288
n_tokens_per_dead_feature_window (millions): 2621.44
We will reset the sparsity calculation 73 times.
Number tokens in sparsity calculation window: 4.10e+06
Using Ghost Grads.
Run name: 24576-L1-8e-05-LR-0.0004-Tokens-3.000e+08
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.004096
Total training steps: 73242
Total wandb updates: 732
n_tokens_per_feature_sampling_window (millions): 524.288
n_tokens_per_dead_feature_window (millions): 2621.44
We will reset the sparsity calculation 73 times.
Number tokens in sparsity calculation window: 4.10e+06
Using Ghost Grads.


In [9]:
for i, sae in enumerate(sparse_autoencoders):
    hyp = sae.cfg
    print(
        f"{i}: Layer {hyp.hook_point_layer}, p_norm {hyp.lp_norm}, alpha {hyp.l1_coefficient}"
    )

0: Layer 0, p_norm 1, alpha 8e-05


In [10]:
# pick which sae you wnat to evaluate. Default is 0
sparse_autoencoder = sparse_autoencoders[0]

TypeError: 'SAEGroup' object is not subscriptable